In [2]:
import pandas as pd
import numpy as np

netID = "jsc9862"
df = pd.read_csv(f"/scratch/{netID}/Explaining_Song_Popularity/final_cleaned_songs.csv")

# Shuffle to balance load (optional but recommended)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into 5 shards
num_shards = 5
shards = np.array_split(df, num_shards)

for i, shard in enumerate(shards):
    out_path = f"/scratch/{netID}/Explaining_Song_Popularity/data/shards/songs_shard_{i}.csv"
    shard.to_csv(out_path, index=False)
    print("Saved:", out_path)

/home/jsc9862/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Saved: /scratch/jsc9862/Explaining_Song_Popularity/data/shards/songs_shard_0.csv
Saved: /scratch/jsc9862/Explaining_Song_Popularity/data/shards/songs_shard_1.csv
Saved: /scratch/jsc9862/Explaining_Song_Popularity/data/shards/songs_shard_2.csv
Saved: /scratch/jsc9862/Explaining_Song_Popularity/data/shards/songs_shard_3.csv
Saved: /scratch/jsc9862/Explaining_Song_Popularity/data/shards/songs_shard_4.csv


We introduce librosa, a music/audio analysis library that computes time–frequency representations for our song previews.

We collected the following features and outputed songs_with_librosa_features.pkl:

- duration (should be 30 seconds)
- rms_energy (root mean square energy across time; higher RMS = louder audio)
- spectral_centroid (“center of mass” of the spectrum; Higher centroid = brighter or sharper sounds)
- spectral_bandwidth (spread of frequencies around centroid; Higher bandwidth = more complex or rich harmonic content)
- spectral_rolloff (frequency below which 85% of the energy is contained)
- zero_crossing_rate (number of times the signal crosses zero amplitude; High ZCR = noisy, sharp, or percussive sound)
- tempo (Estimated BPM)
- chroma_stft_mean (Average chroma vector from short-time Fourier transform; Represents pitch class distribution (C, C#, D, ...)
- chroma_cqt_mean (Chroma from constant-Q transform; Captures harmonic structure & tonality)
- chroma_cens_mean (Chroma with energy normalization; Good for similarity & retrieval tasks)
- mfcc_mean_1 - mfcc_mean_13 (Average Mel-Frequency Cepstral Coefficients describe overall timbre)
- mfcc_std_1 … mfcc_std_13 (Standard deviation of MFCCs; Measures how much timbre varies over time
- spectral_contrast (Difference between peaks and valleys in each frequency band; Higher contrast = strong percussive and rhythmic sounds)
- tonnetz (Tonal centroid features; capture key, mode, and harmonic progression tendencies)

In [16]:
import pandas as pd
netID="jsc9862"

shard1 = pd.read_pickle(f"/scratch/jsc9862/Explaining_Song_Popularity/data/librosa_shard_0.pkl")
shard2 = pd.read_pickle(f"/scratch/{netID}/Explaining_Song_Popularity/data/librosa_shard_1.pkl")
shard3 = pd.read_pickle(f"/scratch/{netID}/Explaining_Song_Popularity/data/librosa_shard_2.pkl")
shard4 = pd.read_pickle(f"/scratch/{netID}/Explaining_Song_Popularity/data/librosa_shard_3.pkl")
shard5 = pd.read_pickle(f"/scratch/{netID}/Explaining_Song_Popularity/data/librosa_shard_4.pkl")

In [17]:
total = len(shard1)+len(shard2)+len(shard3)+len(shard4)+len(shard5)

print(total)

20740


In [ ]:
from tqdm import tqdm

feature_dicts = []

for path in tqdm(songs_df["file_path"].tolist(), desc="Extracting librosa features"):
    feature_dicts.append(extract_librosa_features(path))

features_df = pd.DataFrame(feature_dicts)


Extracting librosa features:   0%|          | 0/20740 [00:00<?, ?it/s]/tmp/ipykernel_5262/2045398973.py:41: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=y, sr=sr, onset_envelope=onset_env)
Extracting librosa features:   3%|▎         | 649/20740 [19:16<11:09:50,  2.00s/it]

In [8]:
songs_with_audio_feats = pd.concat([songs_df.reset_index(drop=True),
                                    features_df.reset_index(drop=True)], axis=1)
songs_with_audio_feats.head()

,duration_sec,rms_mean,rms_std,centroid_mean,centroid_std,bandwidth_mean,bandwidth_std,rolloff_mean,rolloff_std,zcr_mean,...,chroma_D_mean,chroma_D#_mean,chroma_E_mean,chroma_F_mean,chroma_F#_mean,chroma_G_mean,chroma_G#_mean,chroma_A_mean,chroma_A#_mean,chroma_B_mean
0,29.975556,0.278993,0.037006,2754.696787,502.869916,2802.366576,291.865969,6146.186641,1044.566481,0.124607,...,0.418044,0.393092,0.338421,0.427980,0.435531,0.408817,0.464590,0.366762,0.457948,0.382962
1,29.929116,0.329715,0.089350,2391.521255,776.926626,2457.443415,388.657153,4981.762506,1518.720013,0.095913,...,0.426955,0.322113,0.446184,0.378618,0.305085,0.318568,0.360675,0.485258,0.388039,0.349617
2,29.929116,0.203293,0.079145,2774.546487,1291.355777,2573.595689,722.648414,5725.451518,2612.383269,0.140336,...,0.463527,0.525835,0.688297,0.560594,0.380335,0.353032,0.321739,0.323214,0.348989,0.472506
3,29.975556,0.253371,0.092798,2535.547324,815.009014,2716.849286,404.152080,5688.018123,1676.584694,0.101630,...,0.399765,0.398603,0.378685,0.387324,0.511542,0.479167,0.426331,0.398629,0.495446,0.457668
4,29.929116,0.076925,0.036367,2280.803290,763.997201,2431.382585,410.807263,4770.506581,1543.145293,0.102507,...,0.303500,0.484029,0.579680,0.355329,0.299414,0.368766,0.544211,0.432613,0.406913,0.468694
5,29.929116,0.150298,0.066227,2298.083292,1085.002901,2299.652503,519.131758,4634.625220,1936.130742,0.096641,...,0.301942,0.524115,0.528028,0.379679,0.357327,0.247962,0.236120,0.182409,0.223931,0.354204
6,29.929116,0.118647,0.032913,2078.214764,786.395399,2257.817314,544.552273,4438.237062,1700.045980,0.083467,...,0.427429,0.395757,0.407153,0.581207,0.459420,0.314073,0.386416,0.502769,0.335647,0.294957
7,29.975556,0.160734,0.032938,1827.704126,583.069284,2108.442134,397.464914,3561.034702,1300.739427,0.086310,...,0.303475,0.366864,0.642310,0.309159,0.229209,0.187991,0.313818,0.330559,0.219774,0.348788
8,29.975556,0.155591,0.047002,1309.966871,680.193498,2016.429989,617.691205,2804.095076,1851.347613,0.035407,...,0.361757,0.302050,0.304178,0.246827,0.291341,0.235128,0.297782,0.421746,0.327225,0.291692
9,29.975556,0.277950,0.111737,2096.134901,882.346507,2408.063825,468.366863,4538.969042,1860.928038,0.082521,...,0.363901,0.417679,0.334494,0.388561,0.465839,0.591883,0.618961,0.448383,0.401340,0.367225


In [ ]:
out_path = f"/scratch/{netID}/Explaining_Song_Popularity/songs_with_librosa_features.pkl"
songs_with_audio_feats.to_pickle(out_path)
